In [1]:
import pandas as pd
import numpy as np
import sys
from pyspark.sql import functions as f
import re
from functools import reduce
from pyspark.sql import SparkSession
from typing import Dict
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import re
from pyspark.sql.window import Window
import plotly.express as px
from pyspark.ml.feature import Bucketizer
from itertools import chain
from pandas.tseries.offsets import MonthEnd
from geopy import distance
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', 200)

In [2]:
#!pip install scikit-learn --force
#!pip install catboost
#!pip install ipywidgets --force

In [3]:
df = pd.read_parquet('prepared_df.parquet')

In [4]:
obj_df = df[df['report_date'] > pd.to_datetime('2019-01-01')].groupby('objectid').apply(lambda df: df.merge(pd.DataFrame(
                {'report_date': pd.date_range(df['report_date'].min() + MonthEnd(-3), df['report_date'].max(), freq = 'M')})).fillna(
                            {col: df[col].mode()[0] if len(df[col].mode()) > 0 else np.nan for col in ['latitude', 'longitude']})).reset_index(drop = True)


obj_df = obj_df.assign(year = obj_df['report_date'].map(lambda dt: dt.year), 
                       quartal = obj_df['report_date'].map(lambda dt: dt.quarter))[['year', 'quartal', 'objectid', 'latitude', 'longitude']]

obj_df = obj_df.drop_duplicates(subset = ['year', 'quartal', 'objectid'])

In [5]:
obj_df.groupby('objectid').agg({'latitude': 'count', 'longitude': 'count'}).query('latitude != longitude')

,latitude,longitude
objectid,,


In [11]:
t = pd.concat([obj_df.sample(1)[['latitude','longitude']].reset_index(drop = True), 
           obj_df.sample(1)[['latitude','longitude']].rename(columns = dict(zip(['latitude','longitude'], ['latitude_part','longtitude_part']))).reset_index(drop = True)], axis = 1)
t

,latitude,longitude,latitude_part,longtitude_part
0,55.54694,37.590116,55.750337,37.70674


In [14]:
def merge_area(df):
    r_Earth = 6372795
    
    get_dist = f.udf(lambda c1, c2: distance.geodesic(c1, c2).m, FloatType())
    df = df.withColumn('dist', 
                get_dist(f.array('latitude', 'longitude'), f.array('latitude_part', 'longtitude_part'))
                      )
    
    df = df.withColumn('dist1', 
                r_Earth*2*f.asin( 
                    (f.sin( (f.radians('latitude_part') - f.radians('latitude')) / 2 )**2 + \
                     f.cos(f.radians('latitude_part'))*f.cos(f.radians('latitude')) * f.sin( (f.radians('longtitude_part') - f.radians('longitude')) / 2 )**2 )**(1/2)
                                )
                      )
        
    return df

merge_area(spark.createDataFrame(t)).collect()

[Row(latitude=55.54694, longitude=37.590116, latitude_part=55.750337, longtitude_part=37.70674, dist=23805.865234375, dist1=23777.68290707195)]

In [5]:
#spark.table('prod_dm_cian.cd_soba_primary').limit(50).toPandas()#.collect()

In [ ]:
def merge_area_price(esc, df):
    df = esc.join(df, on = ['year', 'quartal'], how = 'left')
    print(df.count())
    r_Earth = 6372795
    
    get_dist = f.udf(lambda c1, c2: distance.geodesic(c1, c2).m, FloatType())
    df = df.withColumn('dist', 
                get_dist(f.array('latitude', 'longitude'), f.array('latitude_part', 'longtitude_part'))
                      )
    #df = df.withColumn('dist', 
    #            r_Earth*2*f.asin(f.sin( (f.radians('latitude_part') - f.radians('latitude')) / 2 )**2 + \
    #                     f.cos(f.radians('latitude_part'))*f.cos(f.radians('latitude')) * f.sin( (f.radians('longtitude_part') - f.radians('longitude')) / 2 )**2 )
    #                  )
    w  = Window.partitionBy('objectid', 'year', 'quartal').orderBy('dist')
    df = df.withColumn('dist_rank', f.dense_rank().over(w))
    df = df.filter(f.col('dist_rank') <= 50)
    return df
dfp = merge_area_price(spark.createDataFrame(obj_df), spark.table('prod_dm_cian.cd_soba_primary')).toPandas()

237851046


/opt/cloudera/parcels/AnacondaPy37/envs/python3.7.9/lib/python3.7/site-packages/pyarrow/util.py:43: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead.
  warnings.warn(msg, FutureWarning)
[Stage 5:=========================================>            (154 + 28) / 200]24/04/16 18:07:27 ERROR cluster.YarnScheduler: Lost executor 17 on p0drp0-db5019xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 3 retries (starting from 61774)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI) to an available port or increasing spark.port.maxRetries.
24/04/16 18:07:28 ERROR cluster.YarnScheduler: Lost executor 19 on p0drp0-db5019xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.networ

In [40]:
dfp.count()

year                        0
quartal                     0
objectid                    0
latitude                    0
longitude                   0
jk_id                       0
jk_nm                       0
latitude_part               0
longtitude_part             0
city                        0
street                      0
house                       0
district                    0
class_nm                    0
rooms_cnt                   0
area_range_part             0
flat_cnt                    0
total_area_part             0
offer_price_sqm_amt         0
offer_currency_price_nm     0
developer_nm                0
currency_price_nm           0
deadline_actual             0
deadline_change             0
deadline_plan               0
count_fl_cnt                0
count_ul_cnt                0
bought_price_sqm_info       0
currency_bought_price_nm    0
source_nm                   0
system_dt                   0
dist                        0
dist_rank                   0
dtype: int

In [33]:
dfp['dist_rank'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
      dtype=int32)

In [10]:
dfp.iloc[:100,:]

,year,quartal,objectid,latitude,longitude,jk_id,jk_nm,latitude_part,longtitude_part,city,street,house,district,class_nm,rooms_cnt,area_range_part,flat_cnt,total_area_part,offer_price_sqm_amt,offer_currency_price_nm,developer_nm,currency_price_nm,deadline_actual,deadline_change,deadline_plan,count_fl_cnt,count_ul_cnt,bought_price_sqm_info,currency_bought_price_nm,source_nm,system_dt,dist,dist_rank
0,2036,2,30390.0,54.663040,39.656265,NaN,None,NaN,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,1
1,2029,2,22768.0,54.614542,39.697427,NaN,None,NaN,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,1
2,2029,2,27498.0,45.063797,39.143150,NaN,None,NaN,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,1
3,2029,2,30390.0,54.663040,39.656265,NaN,None,NaN,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,1
4,2019,1,1802.0,56.919074,53.261569,1520737.0,None,56.91979,53.26272,Ижевск,Переулок выставочный,10,Индустриальный,None,1,от 30 до 50,1.0,36.000000,37778.000000,rur,None,None,None,None,None,NaN,NaN,None,None,ЦИАН,2021-11-25,0.000881,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019,1,1802.0,56.919074,53.261569,51178.0,Норвежский лес,56.87181,53.24156,Ижевск,Улица 10 лет октября,дом № 8,Индустриальный,комфорт,3,от 50 до 70,3.0,60.866667,53670.666667,rur,Комосстрой®,внутренние отделочные работы,2020-10-01,2020-10-01,2020-10-01,NaN,NaN,None,None,ЦИАН,2021-11-25,2.284201,38
96,2019,1,1802.0,56.919074,53.261569,48534.0,Танго,56.87181,53.24156,Ижевск,Улица 10 лет октября,танго,Индустриальный,комфорт,2,от 30 до 50,1.0,44.500000,50431.000000,rur,Уралдомстрой ск,внутренние отделочные работы,2020-04-01,"2020-04-01, 2019-10-01",2020-04-01,NaN,NaN,None,None,ЦИАН,2021-11-25,2.284201,38
97,2019,1,1802.0,56.919074,53.261569,48534.0,Танго,56.87181,53.24156,Ижевск,Улица 10 лет октября,танго,Индустриальный,комфорт,2,от 50 до 70,2.0,55.400000,50111.500000,rur,Уралдомстрой ск,внутренние отделочные работы,2020-04-01,"2020-04-01, 2019-10-01",2020-04-01,NaN,NaN,None,None,ЦИАН,2021-11-25,2.284201,38
98,2019,1,1802.0,56.919074,53.261569,38335.0,Покровский,56.87181,53.24156,Ижевск,Улица 10 лет октября,1,Индустриальный,комфорт,1,от 30 до 50,3.0,33.733333,49899.333333,rur,Комосстрой®,сдан ГК,2019-07-01,2019-07-01,2019-07-01,NaN,NaN,None,None,ЦИАН,2021-11-25,2.284201,38
